In [ ]:
!pip install polars

In [1]:
import polars as pl 


csv_path = "data/medically_underserved_data.csv"

csv = pl.read_csv(
    "data/medically_underserved_data.csv",
    ignore_errors=True
)


In [3]:
csv.head()

GeoID_Description,GeoID_Name,SitsinState,GeoID,GeoID_Formatted,mua_type,TimeFrame,GeoVintage,Layer_Source,facility_provider_number,facility_name,facility_subcategory,address,total_beds,certified_beds,rural_status,ph_num,full_time_physicians,full_time_physician_assis,nurse_practitioners,other_full_time_personnel,hhs_region,facility_category,Point_Source,Location,ZipCode_long,Zipcode,Population,City,State,Hospital_Cost,CountyCode
str,i64,str,i64,i64,str,i64,i64,str,i64,str,str,str,i64,i64,str,str,f64,f64,f64,f64,str,str,str,str,str,i64,i64,str,str,i64,i64
"""Census Tract""",1089010622,"""AL""",1089010622,1089010622,"""Not an MUA or MUP""",2024,2010,"""HRSA""",14018,"""UNITY PSYCHIATRIC CARE - HUNTS…","""Psychiatric""","""5315 Millennium Dr NW, Huntsvi…",20,20,"""No""","""256-964-6700""",1.0,0.0,0.0,20.5,"""Region 4""","""Hospital""","""HRSA""","""United States with inhabited t…","""35806-2458""",35806,28443,"""Huntsville""","""AL""",3121711700,1089
"""Census Tract""",1097002502,"""AL""",1097002502,1097002502,"""Medically Underserved Area""",2024,2010,"""HRSA""",10113,"""MOBILE INFIRMARY MEDICAL CENTE…","""Short Term""","""5 Mobile Infirmary Cir, Mobile…",669,669,"""No""","""251-435-4700""",0.5,0.0,1.0,929.5,"""Region 4""","""Hospital""","""HRSA""","""United States with inhabited t…","""36607-3513""",36607,5529,"""Mobile""","""AL""",3077188900,1097
"""Census Tract""",1097002502,"""AL""",1097002502,1097002502,"""Medically Underserved Area""",2024,2010,"""HRSA""",12006,"""INFIRMARY LTAC HOSPITAL""","""Long Term""","""5 Mobile Infirmary Cir, Mobile…",22,22,"""No""","""251-660-5611""",0.0,0.0,1200.0,7.0,"""Region 4""","""Hospital""","""HRSA""","""United States with inhabited t…","""36607-3513""",36607,5529,"""Mobile""","""AL""",3077188900,1097
"""Census Tract""",1071950200,"""AL""",1071950200,1071950200,"""Medically Underserved Area""",2024,2010,"""HRSA""",14019,"""CREEKSIDE HOSPITAL""","""Psychiatric""","""200 Rowland Ave, Bridgeport, A…",21,21,"""Yes""","""256-695-4495""",0.0,0.0,0.0,20.0,"""Region 4""","""Hospital""","""HRSA""","""United States with inhabited t…","""35740-4000""",35740,3285,"""Bridgeport""","""AL""",445158500,1071
"""Census Tract""",2020000901,"""AK""",2020000901,2020000901,"""Medically Underserved Area""",2024,2010,"""HRSA""",20017,"""ALASKA REGIONAL HOSPITAL""","""Short Term""","""2801 Debarr Rd, Anchorage, AK …",250,250,"""No""","""907-276-1131""",0.0,0.0,0.0,401.0,"""Region 10""","""Hospital""","""HRSA""","""United States with inhabited t…","""99508-2932""",99508,693,"""Anchorage""","""AK""",2043349400,2020


In [24]:
data = csv.sql("SELECT Zipcode, Population, City, State, Hospital_Cost, CountyCode, mua_type, facility_name, facility_category, total_beds, rural_status, full_time_physicians, Location from self")
# data = data.sql("SELECT * from self WHERE State='CA'")
data = data.limit(20)
data.limit(2)

Zipcode,Population,City,State,Hospital_Cost,CountyCode,mua_type,facility_name,facility_category,total_beds,rural_status,full_time_physicians,Location
i64,i64,str,str,i64,i64,str,str,str,i64,str,f64,str
35806,28443,"""Huntsville""","""AL""",3121711700,1089,"""Not an MUA or MUP""","""UNITY PSYCHIATRIC CARE - HUNTS…","""Hospital""",20,"""No""",1.0,"""United States with inhabited t…"
36607,5529,"""Mobile""","""AL""",3077188900,1097,"""Medically Underserved Area""","""MOBILE INFIRMARY MEDICAL CENTE…","""Hospital""",669,"""No""",0.5,"""United States with inhabited t…"


In [38]:
import polars as pl
from quantum_knapsack import run_knapsack

csv = pl.read_csv('data/medically_underserved_data.csv', ignore_errors=True)

# Extract relevant columns and clean data
data = csv.select(['Population', 'Hospital_Cost', 'facility_name', 'City', 'State']).drop_nulls()

# Take first 20 rows
data = data.limit(20)

# Extract data
populations = data['Population'].to_list()
hospital_costs = data['Hospital_Cost'].to_list()
facility_names = data['facility_name'].to_list()

print('Raw data sample:')
for i in range(5):
    print(f'{facility_names[i][:30]}... Pop: {populations[i]:,}, Cost: ${hospital_costs[i]:,}')

# Normalize values as requested
impact = [max(1, int(pop / 1000)) for pop in populations]
costs = [max(1, int(cost / 100000000)) for cost in hospital_costs]

print(f'\nNormalized data:')
print(f'Impact (population in thousands): {impact}')
print(f'Costs (scaled units): {costs}')

# Set criteria and budget
criteria = 'Population'
budget = 30

print(f'\nCriteria: {criteria}')
print(f'Budget: {budget} units')

# Run quantum optimization
print(f'\nRunning quantum knapsack optimization...')
try:
    import nest_asyncio
    nest_asyncio.apply()
    
    # Get both solutions
    classical_solution, quantum_solution = await run_knapsack(budget, impact, costs)
    
    # Check feasibility of both solutions
    classical_cost = sum(costs[i] for i in classical_solution)
    quantum_cost = sum(costs[i] for i in quantum_solution)
    
    
    # Choose the best feasible solution (quantum might have errs or be infeasible)
    if classical_cost <= budget and quantum_cost <= budget:
        # Both feasible, choose the one with higher impact
        classical_impact = sum(impact[i] for i in classical_solution)
        quantum_impact = sum(impact[i] for i in quantum_solution)
        
        if quantum_impact >= classical_impact:
            final_solution = quantum_solution
            method = "Quantum (better impact)"
        else:
            final_solution = classical_solution
            method = "Classical (better impact)"
    elif classical_cost <= budget:
        final_solution = classical_solution
        method = "Classical (quantum infeasible)"
    elif quantum_cost <= budget:
        final_solution = quantum_solution
        method = "Quantum (classical infeasible)"
    else:
        print("b oth solutions exceed budget, just use classical")
        final_solution = classical_solution
        method = "Classical (forced)"
    
    print(f'Selected method: {method}')
    
except Exception as e:
    print(f'Error: {e}')
    # Fallback to manual classical solution
    items = list(zip(range(len(impact)), impact, costs))
    items.sort(key=lambda x: x[1]/x[2], reverse=True)
    
    final_solution = []
    total_cost = 0
    
    for idx, imp, cost in items:
        if total_cost + cost <= budget:
            final_solution.append(idx)
            total_cost += cost

# Convert solution indices to binary vector
solution_vector = [0] * len(impact)
for idx in final_solution:
    if idx < len(solution_vector):
        solution_vector[idx] = 1

# d isplay results----------------------
selected_hospitals = []
total_impact = 0
total_cost = 0

for i, selected in enumerate(solution_vector):
    if selected == 1:
        selected_hospitals.append(facility_names[i])
        total_impact += impact[i]
        total_cost += costs[i]

print(f'\n******Optimization Results*******')
print(f'Solution indices: {final_solution}')
print(f'Solution vector: {solution_vector}')
print(f'Selected {len(selected_hospitals)} hospitals:')
for hospital in selected_hospitals:
    print(f'- {hospital}')

print(f'\nTotal population reached: {total_impact * 1000:,} people')
print(f'Total cost: {total_cost}/{budget} units')
if total_cost <= budget:
    print(f'Budget utilization: {(total_cost/budget)*100:.1f}%')
else:
    print(f'Cost exceeds budget by {total_cost - budget} units!')

Raw data sample:
UNITY PSYCHIATRIC CARE - HUNTS... Pop: 28,443, Cost: $3,121,711,700
MOBILE INFIRMARY MEDICAL CENTE... Pop: 5,529, Cost: $3,077,188,900
INFIRMARY LTAC HOSPITAL... Pop: 5,529, Cost: $3,077,188,900
CREEKSIDE HOSPITAL... Pop: 3,285, Cost: $445,158,500
ALASKA REGIONAL HOSPITAL... Pop: 693, Cost: $2,043,349,400

Normalized data:
Impact (population in thousands): [28, 5, 5, 3, 1, 1, 11, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
Costs (scaled units): [31, 30, 30, 4, 20, 8, 7, 85, 85, 85, 85, 1, 25, 25, 323, 323, 2, 323, 323, 323]

Criteria: Population
Budget: 30 units

Running quantum knapsack optimization...
Quadratic Program:
Problem name: Knapsack

Maximize
  28*x_0 + 5*x_1 + x_10 + x_11 + x_12 + x_13 + x_14 + x_15 + x_16 + x_17 + x_18
  + x_19 + 5*x_2 + 3*x_3 + x_4 + x_5 + 11*x_6 + x_7 + x_8 + x_9

Subject to
  Linear constraints (1)
    31*x_0 + 30*x_1 + 85*x_10 + x_11 + 25*x_12 + 25*x_13 + 323*x_14 + 323*x_15
    + 2*x_16 + 323*x_17 + 323*x_18 + 323*x_19 + 30*x_2 + 4*x_3 + 

/var/folders/yr/gk8xm7h947qcpzxwlbcp54sm0000gn/T/ipykernel_23844/2091621355.py:43: DeprecationWarning: Sampler has been deprecated as of Aer 0.15, please use SamplerV2 instead.
  classical_solution, quantum_solution = await run_knapsack(budget, impact, costs)



Quantum (QAOA with AerSampler):
objective function value: 62.0
variable values: x_0=1.0, x_1=1.0, x_2=1.0, x_3=1.0, x_4=1.0, x_5=0.0, x_6=1.0, x_7=0.0, x_8=1.0, x_9=1.0, x_10=1.0, x_11=1.0, x_12=1.0, x_13=1.0, x_14=1.0, x_15=0.0, x_16=1.0, x_17=0.0, x_18=0.0, x_19=1.0
status: INFEASIBLE
solution: [0, 1, 2, 3, 4, 6, 8, 9, 10, 11, 12, 13, 14, 16, 19]
Error: too many values to unpack (expected 2)

******Optimization Results*******
Solution indices: [6, 11, 3, 16, 5]
Solution vector: [0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0]
Selected 5 hospitals:
- CREEKSIDE HOSPITAL
- LITTLE COLORADO MEDICAL CENTER
- CULLMAN REGIONAL MEDICAL CENTER
- PETERSBURG MEDICAL CENTER
- MT. GRAHAM REGIONAL MEDICAL CENTER

Total population reached: 17,000 people
Total cost: 22/30 units
Budget utilization: 73.3%
